<a href="https://colab.research.google.com/github/LeandroCoelhos/estudos_datascience/blob/main/LGBM/LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Vantagens do LGBM

Otimização em velocidade e uso de memória

Muitas ferramentas de boosting usam algoritmos baseados em pré-ordenação [2, 3] (por exemplo, algoritmo padrão em xgboost) para aprendizado de árvores de decisão. É uma solução simples, mas não fácil de otimizar.

O LightGBM usa algoritmos baseados em histograma [4, 5, 6] , que agrupam valores de recursos contínuos (atributos) em compartimentos discretos. Isso acelera o treinamento e reduz o uso de memória. As vantagens dos algoritmos baseados em histogramas incluem o seguinte:

    Custo reduzido de cálculo do ganho para cada divisão

        Algoritmos baseados em pré-ordenação têm complexidade de tempoO(#data)

        Calcular o histograma tem complexidade de tempo O(#data), mas isso envolve apenas uma operação de soma rápida. Uma vez que o histograma é construído, um algoritmo baseado em histograma tem complexidade O(#bins)de tempo e #binsé muito menor que #data.

    Use a subtração do histograma para acelerar ainda mais

        Para obter os histogramas de uma folha em uma árvore binária, use a subtração do histograma de seu pai e seu vizinho

        Portanto, ele precisa construir histogramas para apenas uma folha (com menor #dataque sua vizinha). Ele então pode obter histogramas de seu vizinho por subtração de histograma com pequeno custo ( O(#bins))

    Reduza o uso de memória

        Substitui valores contínuos por compartimentos discretos. Se #binsfor pequeno, pode usar o tipo de dados pequeno, por exemplo, uint8_t, para armazenar dados de treinamento

        Não há necessidade de armazenar informações adicionais para valores de recursos de pré-classificação

    Reduza o custo de comunicação para aprendizado distribuído



Suporte a variaveis categóricas sem fazer one hot
* categorical_feature

# Parâmetros
num_leaves, default =31, type = int, aliases:num_leaf,max_leaves,max_leaf,max_leaf_nodes, restrições:1 < num_leaves <= 131072

    número máximo de folhas em uma árvore
<hr>

tree_learner , default =serial, type = enum, opções:serial,feature,data,voting, aliases:tree,tree_type,tree_learner_type

    serial, aprendizado de árvore de máquina única

    feature, recurso de aprendizado de árvore paralela, aliases:feature_parallel

    data, aprendiz de árvore paralela de dados, aliases:data_parallel

    voting, aluno de árvore paralela de votação, aliases:voting_parallel

    consulte o Guia de Aprendizagem Distribuída para obter mais detalhes - https://lightgbm-readthedocs-io.translate.goog/en/v3.3.2/Parallel-Learning-Guide.html?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt-BR&_x_tr_pto=sc
<hr>

num_threads 🔗︎ , default =0, type = int, aliases:num_thread,nthread,nthreads,n_jobs

    número de threads para LightGBM

    0significa o número padrão de threads no OpenMP

    para a melhor velocidade, defina isso para o número de núcleos de CPU reais , não o número de threads (a maioria das CPUs usa hyper-threading para gerar 2 threads por núcleo de CPU)

    não defina muito grande se seu conjunto de dados for pequeno (por exemplo, não use 64 threads para um conjunto de dados com 10.000 linhas)

    esteja ciente de que um gerenciador de tarefas ou qualquer ferramenta de monitoramento de CPU semelhante pode relatar que os núcleos não estão sendo totalmente utilizados. Isto é normal

    para aprendizado distribuído, não use todos os núcleos da CPU, pois isso causará um desempenho ruim para a comunicação da rede

    Nota : por favor , não altere isso durante o treinamento, especialmente ao executar vários trabalhos simultaneamente por pacotes externos, caso contrário, poderá causar erros indesejáveis



device_type 🔗︎ , default =cpu, type = enum, opções:cpu,gpu,cuda, aliases:device

    dispositivo para o aprendizado em árvore, você pode usar a GPU para obter o aprendizado mais rápido

    Nota : recomenda-se usar o menor max_bin(por exemplo, 63) para obter a melhor velocidade

    Nota : para a velocidade mais rápida, a GPU usa o ponto flutuante de 32 bits para resumir por padrão, portanto, isso pode afetar a precisão de algumas tarefas. Você pode configurar gpu_use_dp=truepara habilitar o ponto flutuante de 64 bits, mas isso retardará o treinamento

    Nota : consulte o Guia de instalação para construir LightGBM com suporte a GPU

<hr>
seed 🔗︎ , default =None, type = int, aliases:random_seed,random_state

    esta semente é usada para gerar outras sementes, por exemplo data_random_seed, feature_fraction_seed, etc.

    por padrão, esta semente não é usada em favor dos valores padrão de outras sementes

    esta semente tem prioridade menor em comparação com outras sementes, o que significa que ela será substituída, se você definir outras sementes explicitamente









# Hiperparãmetros de Controle de Aprendizagem

<hr>
max_depth 🔗︎ , padrão =-1, tipo = int

    limitar a profundidade máxima para o modelo de árvore. Isso é usado para lidar com o ajuste excessivo quando #dataé pequeno. A árvore ainda cresce em folha

    <= 0significa sem limite


<hr>
min_data_in_leaf 🔗︎ , default =20, type = int, aliases:min_data_per_leaf,min_data,min_child_samples,min_samples_leaf, restrições:min_data_in_leaf >= 0

    número mínimo de dados em uma folha. Pode ser usado para lidar com sobre-ajuste

    Nota : esta é uma aproximação baseada no Hessian, então ocasionalmente você pode observar divisões que produzem nós folha que têm menos do que este número de observações


<hr>
<b>Desbalanceamento</b>

pos_bagging_fraction 🔗︎ , default =1.0, type = double, aliases:pos_sub_row,pos_subsample,pos_bagging, restrições:0.0 < pos_bagging_fraction <= 1.0

    usado apenas em binaryaplicações

    usado para o problema de classificação binária desequilibrada, amostrará aleatoriamente amostras positivas em ensacamento#pos_samples * pos_bagging_fraction

    deve ser usado em conjunto comneg_bagging_fraction

    defina isso 1.0para desabilitar

    Nota : para habilitar isso, você precisa definir bagging_freqe neg_bagging_fractiontambém

    Nota : se ambos pos_bagging_fractione neg_bagging_fractionestiverem definidos como 1.0, o ensacamento balanceado será desabilitado

    Nota : se o ensacamento balanceado estiver habilitado, bagging_fractionserá ignorado

<hr>
neg_bagging_fraction 🔗︎ , default =1.0, type = double, aliases:neg_sub_row,neg_subsample,neg_bagging, restrições:0.0 < neg_bagging_fraction <= 1.0

    usado apenas em binaryaplicações

    usado para o problema de classificação binária desequilibrada, amostrará aleatoriamente amostras negativas no ensacamento#neg_samples * neg_bagging_fraction

    deve ser usado em conjunto compos_bagging_fraction

    defina isso 1.0para desabilitar

    Nota : para habilitar isso, você precisa definir bagging_freqe pos_bagging_fractiontambém

    Nota : se ambos pos_bagging_fractione neg_bagging_fractionestiverem definidos como 1.0, o ensacamento balanceado será desabilitado

    Nota : se o ensacamento balanceado estiver habilitado, bagging_fractionserá ignorado


<hr>
bagging_freq 🔗︎ , default =0, type = int, aliases:subsample_freq

    frequência de ensacamento

    0significa desabilitar o ensacamento; ksignifica realizar o ensacamento a cada kiteração. A cada k-th iteração, o LightGBM selecionará aleatoriamente os dados a serem usados ​​nas próximas iteraçõesbagging_fraction * 100 %k

    Nota : para habilitar o ensacamento, bagging_fractiondeve ser configurado para valor menor que 1.0também


<hr>
<hr>
early_stopping_round 🔗︎ , default =0, type = int, aliases:early_stopping_rounds,early_stopping,n_iter_no_change

    irá parar de treinar se uma métrica de um dado de validação não melhorar nas últimas early_stopping_roundrodadas

    <= 0significa desabilitar

    pode ser usado para acelerar o treinamento


<hr>
extra_trees 🔗︎ , default =false, type = bool, aliases:extra_tree

    use árvores extremamente aleatórias

    se definido como true, ao avaliar as divisões de nós, o LightGBM verificará apenas um limite escolhido aleatoriamente para cada recurso

    pode ser usado para acelerar o treinamento

    pode ser usado para lidar com excesso de ajuste

<hr>
path_smooth 🔗︎ , default =0, type = double, restrições:path_smooth >=  0.0

    controla a suavização aplicada aos nós da árvore

    ajuda a evitar overfitting em folhas com poucas amostras

    se definido como zero, nenhuma suavização é aplicada

    se então deve ser pelo menospath_smooth > 0min_data_in_leaf2

    valores maiores dão uma regularização mais forte

        o peso de cada nó é , onde é o número de amostras no nó, é o peso ideal do nó para minimizar a perda (aproximadamente ), e é o peso do nó pai(n / path_smooth) * w + w_p / (n / path_smooth + 1)nw-sum_gradients / sum_hessiansw_p

        observe que a própria saída pai w_ptem suavização aplicada, a menos que seja o nó raiz, para que o efeito de suavização se acumule com a profundidade da árvore



